In [284]:
import pandas as pd
import spacy
import numpy as np
from spacy import displacy

In [12]:
nlp = spacy.load('../data/models/spacy_model_250421/')

In [89]:
df = pd.read_excel('../data/processed/inland.xlsx')

In [90]:
df.rename(columns = {'Unnamed: 0':'id'}, inplace = True)
df['LOC'] = False
df['DAT'] = False
df['WEA'] = False

In [91]:
df = df[df.text_len <= 15000]

In [92]:
df

,id,date,year,month,day,pub,head,full_text,href,text_len,w_count,sturm,hagel,überschwemmung,LOC,DAT,WEA
0,2247,1828.02.07,1828,2,7,Rigasche Zeitung,Inland.,"Inland.\n\nOdessa, den 14. Januar.\n\nDas Eis,...",http://periodika.lv/viewerOpener?issue=/p_003_...,3664,492,True,False,False,False,False,False
1,2283,1828.04.03,1828,4,3,Rigasche Zeitung,Inland.,"Inland.\n\nRiga, den 23. Marz.\n\nSeine Kaiser...",http://periodika.lv/viewerOpener?issue=/p_003_...,2314,328,False,False,False,False,False,False
3,5392,1845.01.10,1845,1,10,Zuschauer,Inland.,"Inland.\n\nRiga, 10. Januar. Einem osficiellen...",http://periodika.lv/viewerOpener?issue=/dzus18...,6306,748,True,False,False,False,False,False
5,5425,1845.02.24,1845,2,24,Zuschauer,Inland.,"Inland.\n\nRiga, 23. Februar. Gestern Abend um...",http://periodika.lv/viewerOpener?issue=/dzus18...,6359,735,True,False,False,False,False,False
6,5438,1845.03.14,1845,3,14,Zuschauer,Inland.,Inland.\n\nRiga. Wir haben die erfreuliche Aus...,http://periodika.lv/viewerOpener?issue=/dzus18...,5447,669,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3123,33171,1898.11.30,1898,11,30,Libausche Zeitung,Inland.,Inland.\n\nApothekenwesen.\n\nD i e Steuer auf...,http://periodika.lv/viewerOpener?issue=/p_003_...,13120,1739,False,False,True,False,False,False
3124,33180,1899.04.08,1899,4,8,Rigasche Rundschau,Inland,"Inland\n\n«_iga, 8. April.\nZur _Hochwassernot...",http://periodika.lv/viewerOpener?issue=/p_001_...,4677,590,False,False,True,False,False,False
3125,33181,1899.04.09,1899,4,9,Libausche Zeitung,Inland.,Inland.\n\nUeber die Ätudenteunnordnungeu.\n5)...,http://periodika.lv/viewerOpener?issue=/p_003_...,13696,1734,False,False,True,False,False,False
3126,33191,1899.06.10,1899,6,10,Libausche Zeitung,Inland,"Inland\n\nZur baltischen Krugs frage,\nwelche ...",http://periodika.lv/viewerOpener?issue=/p_003_...,6808,882,False,False,True,False,False,False


In [149]:
doc = nlp(df.iloc[1003].full_text)

In [150]:
displacy_color_code = {'WEA': '#4cafd9',
                  'PER': '#ffb366',
                  'DAT': '#bf80ff',
                  'LOC': '#a88676',
                  'MISC': 'grey',
                  'MEA': '#85e085',
                  'ORG': '#5353c6'}

displacy_options = {'ents': ['WEA', 'PER', 'DAT', 'LOC', 'MISC', 'MEA', 'ORG'], 'colors': displacy_color_code}

displacy.render(doc, style='ent', jupyter=True, options=displacy_options)

In [255]:
ee = pd.read_csv('../data/external/geonames/EE.tsv', sep='\t', header=None)
ee.columns = ['id', 'name', 'asciiname', 'altnames',
              'lat', 'long', 'feature_class', 'feature_code',
              'country_code', 'cc2', 'admin1', 'admin2',
              'admin3', 'admin4', 'pop', 'elevation',
              'dem', 'timezone', 'modification_date']
#ee.head()

In [256]:
lv = pd.read_csv('../data/external/geonames/LV.tsv', sep='\t', header=None)
lv.columns = ['id', 'name', 'asciiname', 'altnames',
              'lat', 'long', 'feature_class', 'feature_code',
              'country_code', 'cc2', 'admin1', 'admin2',
              'admin3', 'admin4', 'pop', 'elevation',
              'dem', 'timezone', 'modification_date']
#lv.head()

In [253]:
ee_altnames = []

for lst in list(ee.altnames.str.split(',')):
    if type(lst) == list:
        for name in lst:
            ee_altnames.append(name)

In [254]:
lv_altnames = []

for lst in list(lv.altnames.str.split(',')):
    if type(lst) == list:
        for name in lst:
            lv_altnames.append(name)

In [677]:
ix = ee[ee.altnames.str.contains('Dorpat') == True]

In [682]:
ix.id.values[0]

588335

In [132]:
import edit_distance
ed = edit_distance.edit_distance

In [458]:
def entity_positions_old(doc):    
    
    ents = []
    inside = False

    for i in range(0, len(doc)-1):

        if doc[i].ent_iob != 2:
            if inside == False:
                ent_start = i
            inside = True

        else:
            if inside == True:
                ent_end = i
                ents.append((ent_start, ent_end))
                inside = False

    return ents

In [459]:
def entity_positions(doc):    
    
    ents = []

    for i in range(0, len(doc)):
        
        if doc[i].ent_iob == 3:
            ents.append(i)

    return ents

In [633]:
def build_concordances(doc, entlabel, entdist, window_before, window_after):
    
    # get the indexes of entities
    positions = entity_positions(doc)
    concordances = []
    
    # leave out all other labels than entlabel
    for i in range(0, len(doc.ents)-1):
        if doc.ents[i].label_ != entlabel:
            positions[i] = 0
            
    positions = [ent for ent in positions if ent != 0]
    used_positions = []
    
    diff = (pos1-pos0 for pos0, pos1 in zip(positions, positions[1:]))
    split_at = [i for i, dif in enumerate(diff, 1) if dif >= entdist]
    ent_groups = [positions[i:j] for i, j in zip([0]+split_at, split_at+[None])]
    ent_groups = [group for group in ent_groups if len(group) > 1]
    
    for group in ent_groups:
                              
        if window_before < group[1]:
            concordance_start = group[1] - window_before
        else:
            concordance_start = 0

        if window_after < (len(doc) - group[-1]):
            concordance_end = group[-1] + window_after
        else:
            concordance_end = len(doc)
                
                
        concordance = doc[concordance_start:concordance_end]
        concordances.append(concordance)
        
    return concordances

In [648]:
def closest_name(string, namelist):
    
    if string in namelist:
        return (string, 0)
    
    else:
        replace_dict = {'ß':'ss'}
        string = string.lower()
        bestname = ''
        bestdist = len(string)

        for name in namelist:
            dist = ed(string, name.lower())[0]
            if dist < bestdist:
                bestname = name
                bestdist = dist
                if dist == 1:
                    break
        
        for key in replace_dict.keys():
            if key in string:
                string = string.replace(key,replace_dict[key])
                
                for name in namelist:
                    dist = ed(string, name.lower())[0]
                    if dist < bestdist:
                        bestname = name
                        bestdist = dist
                        if dist == 0:
                            break
                    
        
        return (bestname, bestdist)

In [639]:
def get_loc_names(doc, ee=ee_altnames, lv=lv_altnames, maxdist=1):
    
    entset = set([ent.text for ent in doc.ents if ent.label_ == 'LOC'])
   
    if len(entset) > 0:
        
        for entname in entset:
            print('Searching for', entname)
            location_ee = closest_name(entname, ee)
            location_lv = closest_name(entname, lv)

            if min(location_ee[1], location_lv[1]) > 1:
                print('No candidates found \n')

            else:
                if location_ee[1] < location_lv[1]:
                    loc_name = location_ee[0]
                else:
                    loc_name = location_lv[0]
                print('Best match:', loc_name, '\n')
                loc_names.append(loc_name)
         
            
        return loc_names
    else:
        return None

In [637]:
samples = df.sample(5)

In [638]:
for i in range(0, len(samples)):
    
    print(samples.iloc[i].id)
    
    doc_places = []
    doc_
    doc = nlp(samples.iloc[i].full_text)
    concs = build_concordances(doc, 'WEA', 30, 30, 30)
    for conc in concs:
        loc_names = get_loc_names(conc)
        print(loc_names)

14830
Searching for Baumwollengarn
No candidates found 

Searching for Sla«
No candidates found 

Searching for Dutch
No candidates found 

Searching for bi«
No candidates found 

Searching for Ueber-
reichung de«
No candidates found 

Searching for Petersburg
No candidates found 

Searching for Gitliftkkit
No candidates found 

[]
13965
Searching for Allerhöchst
No candidates found 

[]
Searching for Moskau
No candidates found 

[]
32420
Searching for Gouvernement
Tambow
No candidates found 

Searching for Pensa
No candidates found 

Searching for Grobin
Best match: Grobin 

Searching for Pernau
Best match: Pernau 

Searching for Fcflin
No candidates found 

Searching for Dorpat
Best match: Dorpat 

Searching for Rußland
No candidates found 

Searching for noworshcwschen Kreises
No candidates found 

Searching for Fellin
Best match: Fellin 

Searching for Nntzcn
No candidates found 

Searching for Rshrw
No candidates found 

Searching for Tarwast
Best match: Tarwast 

Searching for Pe

In [651]:
closest_name('Weisenstein', ee_altnames)

('Weissenstein', 1)

In [513]:
testdoc = nlp(df[df.id == 15691].full_text.values[0])

In [608]:
concs = build_concordance2(testdoc,'WEA',35,30,30)

In [609]:
for conc in concs:
    print(conc)
    print('--------------------')

Wie die »Latw. awiseS.' meldet, hat am
25. Juni c. innerhalb einer Strecke von 4 Werst in Neu-Nahden
ein zerstörender, heftiger Sturm mit Hagelschlag stattgefunden,
welcher 18 Gesindswirthen in Neu • Rahden an ihren Feldern
und Fensterscheiben einen empfindlichen Schaden zugefügt hat.
Aus der Sehodemehen Umgegend in Kurland
--------------------
der Sehodemehen Umgegend in Kurland, 1. August.
Mit wenigen Unterbrechungen fiel vom 15.—27. Juli fast
täglich Gewitterregen. Am 23. entlud sich ein recht heftiges
Gewitter in hiesiger Gegend und der Blitz schlug in ein lithaui»
sches Häuschen, bei Truiken in Lithauen, ein und äscherte selbigeE
nieder. Auch an andern Orten in Kurland soll der Blitz am
selben Tage eingeschlagen haben. — Das Jahr 1877 ist bezüg
lich des Reifwerdens des Roggens ein Gegensatz zu
Jahre geworden. Damals — im
--------------------
.Nord. B.' geschrieben wird, noch in Nikolajew, um seine Re-
paraturen zu beendigen. Ueber das türkische Panzerschiff, das
beim Kampfe mit de

In [607]:
displacy.render(testdoc, style='ent', jupyter=True, options=displacy_options)

In [497]:
for conc in concs:    
    displacy.render(conc, style='ent', jupyter=True, options=displacy_options)

In [559]:
entlist = entity_positions(testdoc)
entlist = [pos for pos in entlist if testdoc.ents[entlist.index(pos)].label_ == 'WEA']

In [563]:
entlist

[33, 35, 80, 90, 96, 126, 1548, 1576, 1603]

In [579]:
dist = 30
lists = []

for i in range(0, len(entlist)-1):
    alist = []
    if entlist[i+1] - entlist[i] <= dist:
        print(f'found 2 ents closeby: {entlist[i]}, {entlist[i+1]}')
        start = entlist[i]
        for first, second in zip(entlist[i+1:], entlist[i+2:]):
            print('checking next ones')
            if second - first <= dist:
                print(f'{second} - {first} is smaller than dist')
                end = second
                print(f'end of chunk is now {end}')
                break
        lists.append((start,end))

lists

found 2 ents closeby: 33, 35
checking next ones
checking next ones
90 - 80 is smaller than dist
end of chunk is now 90
found 2 ents closeby: 80, 90
checking next ones
96 - 90 is smaller than dist
end of chunk is now 96
found 2 ents closeby: 90, 96
checking next ones
126 - 96 is smaller than dist
end of chunk is now 126
found 2 ents closeby: 96, 126
checking next ones
checking next ones
1576 - 1548 is smaller than dist
end of chunk is now 1576
found 2 ents closeby: 1548, 1576
checking next ones
1603 - 1576 is smaller than dist
end of chunk is now 1603
found 2 ents closeby: 1576, 1603


[(33, 90), (80, 96), (90, 126), (96, 1576), (1548, 1603), (1576, 1603)]

In [596]:
entlist = [33, 35, 80, 90, 96, 126, 1548, 1576, 1603]

diff = (pos1-pos0 for pos0, pos1 in zip(entlist, entlist[1:]))
split_at = [i for i, dif in enumerate(diff, 1) if dif >= 30]
groups = [entlist[i:j] for i, j in zip([0]+split_at, split_at+[None])]

ent_groups

[[33, 35], [80, 90, 96], [126], [1548, 1576, 1603]]

In [594]:
next(enumerate(diff,1))

StopIteration: 

In [590]:
groups

[[33, 35], [80, 90, 96], [126], [1548, 1576, 1603]]